======================================\
Graded Challange 3

Nama: Yobi Aditias\
Phase: 0\
Program : FTDS\
Lok: Pondok Indah\
======================================\

In [43]:
# proses import package yang diperlukan
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import requests
import time
import numpy as np
from scipy import stats

# A. Web Scraping

In [14]:
driver = webdriver.Chrome()

url="https://www.tokopedia.com/search?navsource=&page=1&q=seblak&srp_component_id=02.01.00.00&srp_page_id=&srp_page_title=&st="
driver.get(url)
html = driver.page_source

soup = BeautifulSoup(html, "html.parser") 
print(soup.prettify()[:700])

<html data-rh="lang,translate" lang="id" translate="no">
 <head>
  <title>
   Jual seblak | Tokopedia
  </title>
  <meta charset="utf-8"/>
  <meta content="initial-scale=1, minimum-scale=1, maximum-scale=5, user-scalable=yes, width=device-width" name="viewport"/>
  <link crossorigin="" href="https://assets.tokopedia.net" rel="preconnect"/>
  <link href="https://assets.tokopedia.net" nonce="" rel="dns-prefetch"/>
  <link crossorigin="" href="https://gql.tokopedia.com" rel="preconnect"/>
  <link href="https://gql.tokopedia.com" nonce="" rel="dns-prefetch"/>
  <link crossorigin="" href="https://googleads.g.doubleclick.net" rel="preconnect"/>
  <link href="https://googleads.g.doubleclick.net" no


In [15]:
list_data = []

for i in range(1,11):
    url="https://www.tokopedia.com/search?navsource=&page={}&q=seblak&srp_component_id=02.01.00.00&srp_page_id=&srp_page_title=&st=".format(i)
    driver.get(url)

    time.sleep(3)

    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")

    time.sleep(1)

    """kelas css-974ipl digunakan sebagai pengikat yang mengambil data-data didalamnya,
    maksudnya ialah supaya masing-masing kelas tidak diambil dalam list masing2
    supaya tidak ada yang tertukar ketika len dari masing2 kelas tidak sama panjang"""
    for item in soup.find_all('div', {"class": "css-974ipl"}):
        dataperkelas = {
            'nama_produk': item.find('div', {"class":"prd_link-product-name css-3um8ox"}).get_text() if item.find('div', {"class":"prd_link-product-name css-3um8ox"}) else None,
            'harga': item.find('div', {"class":"prd_link-product-price css-1ksb19c"}).get_text() if item.find('div', {"class":"prd_link-product-price css-1ksb19c"}) else None,
            'nama_toko': item.find('span', {"class": "prd_link-shop-name css-1kdc32b flip"}).get_text() if item.find('span', {"class": "prd_link-shop-name css-1kdc32b flip"}) else None,
            'lokasi': item.find('span', {"class": "prd_link-shop-loc css-1kdc32b flip"}).get_text() if item.find('span', {"class": "prd_link-shop-loc css-1kdc32b flip"}) else None,
            'total_terjual': item.find('span', {"class": "prd_label-integrity css-1duhs3e"}).get_text() if item.find('span', {"class": "prd_label-integrity css-1duhs3e"}) else None,
            'rating': item.find('span', {"class": "prd_rating-average-text css-t70v7i"}).get_text() if item.find('span', {"prd_rating-average-text css-t70v7i"}) else None
        }
        list_data.append(dataperkelas)

datatokped = pd.DataFrame(list_data)


Pada tahapan ini, didapati bahwa toko yang berada dalam kolom iklan tidak memiliki lokasi dan nama toko dalam kelas css-974ipl. Sehingga dalam menyingkirkan data produk yang berada pada kolom iklan dilakukan saat data cleaning.

In [17]:
datatokped.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 374 entries, 0 to 373
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   nama_produk    374 non-null    object
 1   harga          374 non-null    object
 2   nama_toko      350 non-null    object
 3   lokasi         350 non-null    object
 4   total_terjual  363 non-null    object
 5   rating         357 non-null    object
dtypes: object(6)
memory usage: 17.7+ KB


Hasil scrapping didapati bahwa jumlah produk toko yang telah di scrape sebanyak 374 yang masih terdapat nilai null disana baik di rating dan total terjual atau pada toko dan lokasinya. pada kasus toko dan lokasi itu mengartikan toko tersebut berada pada bagian iklan yang nanti akan dibersihkan karena nama toko dan lokasinya tidak berada di dalam kotak class "css-974ipl". Sedangkan ketidak adaan dari rating ataupun total terjual menandakan toko tersebut masih baru dan akan dihilangkan dalam data cleaning

In [18]:
datatokped.to_csv("Data_Scraping.csv", index=False)
"""digunakan untuk menyimmpan data 
supaya dapat dikerjakan lagi ketika vscode ditutup.
saya beri (1) bila ingin melakukan run dari awal
maka csv yang sudah ada tidak tertimpa"""

'digunakan untuk menyimmpan data \nsupaya dapat dikerjakan lagi ketika vscode ditutup'

In [44]:
datascrap = pd.read_csv("Data_Scraping.csv")
"""datascrap merupakan variabel yang selanjutnya akan 
digunakan sebagai data diolah
Selain itu dilakukan pengamanan ekstra dengan mengcopy file
Data_Scraping_GC3.csv ke folder lain"""

'datascrap merupakan variabel yang selanjutnya akan \ndigunakan sebagai data diolah\nSelain itu dilakukan pengamanan ekstra dengan mengcopy file\nData_Scraping_GC3.csv ke folder lain'

# B. Data Preparation

### 1. Lakukan eksplorasi data sederhana:

- Tampilkan beberapa baris data - tuliskan insight
- Tampilkan informasi rangkuman data - tuliskan insight dan berikan penjelasan langkah apa yang akan kamu lakukan
- Cek missing value - tuliskan insight dari temuan yang kamu dapatkan

In [5]:
datascrap

,nama_produk,harga,nama_toko,lokasi,total_terjual,rating
0,SEBLAK JUARA INSTAN MASAK BASAH ASLI BANDUNG E...,Rp 22.000,NaN,NaN,1 rb+ terjual,4.9
1,Seblak Juara ! Seblak Instan Rasa Original - 2...,Rp 115.000,NaN,NaN,2 rb+ terjual,4.9
2,CUANKIE JUARA! CUANKIE JAJANAN KULINER ENAK KH...,Rp 22.000,NaN,NaN,250+ terjual,4.9
3,seblak instan jeletot jajanan hits bandung 1 p...,Rp4.180,tokomakanankalimantan,Jakarta Barat,1 rb+ terjual,4.8
4,"Kerupuk Seblak Rafael Viral - PEDAS DJ, 125 GRAM",Rp11.640,Golden Cakery,Jakarta Pusat,25 terjual,4.8
...,...,...,...,...,...,...
369,Seblak Instan Pedas Home Made,Rp3.500,the Dhecip,Tangerang Selatan,2 rb+ terjual,4.8
370,KERUPUK PEDAS KRIWIL BANTAT ISI 250gr KRITING ...,Rp17.248,AHR Jaya Snack,Jakarta Timur,20 terjual,5.0
371,SEBRING KRUPUK KERUPUK SEBLAK KERING PEDAS DAU...,Rp16.000,Aydaa Snack,Surakarta,40+ terjual,4.9
372,Seblak Instan Komplit Pedas Gurih Nikmat,Rp5.999,Baso Aci Ayang,Kab. Garut,2 rb+ terjual,4.9


In [6]:
datascrap.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 374 entries, 0 to 373
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   nama_produk    374 non-null    object 
 1   harga          374 non-null    object 
 2   nama_toko      350 non-null    object 
 3   lokasi         350 non-null    object 
 4   total_terjual  363 non-null    object 
 5   rating         357 non-null    float64
dtypes: float64(1), object(5)
memory usage: 17.7+ KB


Dalam eksplorasi menampilkan data dan menggunakan fungsi .info() didapati terdapat 374 data toko yang masing-masing masih dalam bentuk string kecuali pada rating yang sudah dalam keaadan float. Sehingga langkah berikutnya ialah membersihkan data yang memiliki NAN karena berupa toko dalam iklan dan toko yang masih baru yang data ini tidak akan digunakan dalam riset lebih lanjut.

### 2. Data Cleaning

handle missing value, mengubah bentuk atau tipe data, perhitungan statistik

In [45]:
cleanNAN = datascrap.dropna()
cleanNAN.info()

<class 'pandas.core.frame.DataFrame'>
Index: 336 entries, 3 to 373
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   nama_produk    336 non-null    object 
 1   harga          336 non-null    object 
 2   nama_toko      336 non-null    object 
 3   lokasi         336 non-null    object 
 4   total_terjual  336 non-null    object 
 5   rating         336 non-null    float64
dtypes: float64(1), object(5)
memory usage: 18.4+ KB


Proses ini menggunakan fungsi .dropna() untuk menghapus data atau baris yang ada nilai null nya didalamnya

In [8]:
cleanNAN

,nama_produk,harga,nama_toko,lokasi,total_terjual,rating
3,seblak instan jeletot jajanan hits bandung 1 p...,Rp4.180,tokomakanankalimantan,Jakarta Barat,1 rb+ terjual,4.8
4,"Kerupuk Seblak Rafael Viral - PEDAS DJ, 125 GRAM",Rp11.640,Golden Cakery,Jakarta Pusat,25 terjual,4.8
6,KERUPUK SEBLAK MENTAH ANEKA WARNA,Rp11.500,jajanangarut19,Jakarta Selatan,100+ terjual,4.9
7,Seblak Instan Ceu Nthien Khas Bandung Rasana N...,Rp15.000,Central Seblak Nusantara,Tangerang Selatan,1 rb+ terjual,5.0
8,seblak frozen,Rp10.400,pelangi girls naget,Jakarta Timur,2 rb+ terjual,4.9
...,...,...,...,...,...,...
369,Seblak Instan Pedas Home Made,Rp3.500,the Dhecip,Tangerang Selatan,2 rb+ terjual,4.8
370,KERUPUK PEDAS KRIWIL BANTAT ISI 250gr KRITING ...,Rp17.248,AHR Jaya Snack,Jakarta Timur,20 terjual,5.0
371,SEBRING KRUPUK KERUPUK SEBLAK KERING PEDAS DAU...,Rp16.000,Aydaa Snack,Surakarta,40+ terjual,4.9
372,Seblak Instan Komplit Pedas Gurih Nikmat,Rp5.999,Baso Aci Ayang,Kab. Garut,2 rb+ terjual,4.9


In [9]:
# untuk mengecek apakah nama produk ada yang sama atau tidak
cleanNAN['nama_produk'].unique

<bound method Series.unique of 3      seblak instan jeletot jajanan hits bandung 1 p...
4       Kerupuk Seblak Rafael Viral - PEDAS DJ, 125 GRAM
6                      KERUPUK SEBLAK MENTAH ANEKA WARNA
7      Seblak Instan Ceu Nthien Khas Bandung Rasana N...
8                                          seblak frozen
                             ...                        
369                        Seblak Instan Pedas Home Made
370    KERUPUK PEDAS KRIWIL BANTAT ISI 250gr KRITING ...
371    SEBRING KRUPUK KERUPUK SEBLAK KERING PEDAS DAU...
372             Seblak Instan Komplit Pedas Gurih Nikmat
373    KERUPUK SEBLAK MINI KENCUR JABLAY BANTET PEDAS...
Name: nama_produk, Length: 336, dtype: object>

dataframe 'cleanNAN' merupakan dataframe setelah dilakukan pembersihan data null. unik value dari kolom nama produk yang jumlahnya sama besar dengan nilai len nya (dilihat dari length yang nilainya 336), maka dianggap tidak ada data duplikat pada data, sehingga dataframe 'cleanNAN' dianggap bisa untuk dilanjutkan.

In [46]:
# dilakukan data copy bertujuan untuk menjaga bila terjadi kesalahan pengodingan
#selain itu juga ditujukan supaya nama variabelnya lebih mudah untuk dibaca oleh programer lain
dfseblak = cleanNAN.copy()

In [47]:
dfseblak['harga'] = dfseblak['harga'].str.replace("Rp","").str.replace(".","")
dfseblak['total_terjual'] = dfseblak['total_terjual'].str.replace("terjual","").str.replace("rb","000").str.replace("+","").str.replace(" ","")
dfseblak = dfseblak.astype({'harga':int ,'total_terjual':int})

In [12]:
dfseblak.info()

<class 'pandas.core.frame.DataFrame'>
Index: 336 entries, 3 to 373
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   nama_produk    336 non-null    object 
 1   harga          336 non-null    int32  
 2   nama_toko      336 non-null    object 
 3   lokasi         336 non-null    object 
 4   total_terjual  336 non-null    int32  
 5   rating         336 non-null    float64
dtypes: float64(1), int32(2), object(3)
memory usage: 15.8+ KB


dua kodingan diatas merupakan upaya dalam menyikapi kolom harga dan total terjual. Dimulai dengan menghapus string yang tidak diperlukan dan mengubahnya menjadi int. Tak lupa string rb diganti dengan angka nol 3

# C. Problem Statmenet

Case: Kamu adalah seorang penjual seblak yang cukup terkenal. Kepopuleran seblak ini terdengar hingga ke pulau-pulau lain di Indonesia. Berhubung pengiriman antar pulau, rata-rata produk yang diterima oleh konsumen luar Pulau Jawa sudah tidak sebagus seperti yang diawal. Hal ini mengakibatkan rating toko kamu turun hingga 3.7.
Akhirnya kamu memutuskan untuk membuat gudang di daerah luar Pulau Jawa. Untuk mempermudah proses distibusi dan harapannya dapat meningkatan customer satisfication

Metode SMART menyesuaikan dengan kondisi simulasi pada case diatas

- SPESIFIC : melakukan peningkatkan rating pelanggan dengan cara meningkatkan satisfication pelanggan diluar Pulau Jawa dengan cara membuka gudang baru diluar Pulau Jawa berdasarkan analisa pesebaran penjualan

- MEASUREABLE : Meningkatkan satisfication rate store dari 3.7 hingga lebih dari 4.5

- ACHIEVEABLE : Hal ini dinilai cukup achieveable berdasarkan data historical rating cukup tinggi sebelum terkenal hingga ke luar pulau jawa, selain itu dari produk-produk lain yang memiliki nilai hingga 4.9 maka target ini dinilai cukup achievable

- RELEVANT : Peningkatan rating itu sangat penting, pada jurnal-jurnal yang sering digunakan dalam bisnis, saat ini masyarakat cenderung lebih mempercayai data berupa UGC (User Generated Content) yang berarti rating dan ulasan dari pelanggan lain lebih cenderung untuk dijadikan tolok ukur bagi pelanggan lainya.

- TIMEBOUND : Dalam waktu satu minggu sudah ditentukan dimana ingin membuka gudang dan sudah menentukan strategi marketing untuk tempat yang baru sesuai dengan hasil analisis, dalam waktu 3 bulan sudah meningkatkan rating toko dari 3.7 menjadi 4.5

# D. Analysis

In [48]:
mean_harga = dfseblak['harga'].mean()
med_harga = dfseblak['harga'].median()
std_harga = dfseblak['harga'].std()
skw_harga = dfseblak['harga'].skew()
kur_harga = dfseblak['harga'].kurtosis()
mean_terjual = dfseblak['total_terjual'].mean()
med_terjual = dfseblak['total_terjual'].median()
std_terjual = dfseblak['total_terjual'].std()
skw_terjual = dfseblak['total_terjual'].skew()
kur_terjual = dfseblak['total_terjual'].kurtosis()
mean_rating = dfseblak['rating'].mean()
med_rating = dfseblak['rating'].median()
std_rating = dfseblak['rating'].std()
skw_rating = dfseblak['rating'].skew()
kur_rating = dfseblak['rating'].kurtosis()

print(f"A. Harga:\nrata-rata = {round(mean_harga, 2)}\nmedian = {round(med_harga, 2)}\nstandar deviasi = {round(std_harga, 2)}\nskewness = {round(skw_harga, 2)}\nkurotsis = {round(kur_harga, 2)}")
print(f"\nB. Jumlah Produk Terjual:\nrata-rata = {round(mean_terjual, 2)}\nmedian = {round(med_terjual, 2)}\nstandar deviasi = {round(std_terjual, 2)}\nskewness = {round(skw_terjual, 2)}\nkurotsis = {round(kur_terjual, 2)}")
print(f"\nC. Rating:\nrata-rata = {round(mean_rating, 2)}\nmedian = {round(med_rating, 2)}\nstandar deviasi = {round(std_rating, 2)}\nskewness = {round(skw_rating, 2)}\nkurotsis = {round(kur_rating, 2)}")


A. Harga:
rata-rata = 14898.32
median = 14000.0
standar deviasi = 12344.81
skewness = 2.93
kurotsis = 13.58

B. Jumlah Produk Terjual:
rata-rata = 1289.05
median = 500.0
standar deviasi = 2421.64
skewness = 3.02
kurotsis = 8.2

C. Rating:
rata-rata = 4.87
median = 4.9
standar deviasi = 0.13
skewness = -2.24
kurotsis = 7.42


standar deviasi pada harga bernilai skitar 12 ribu yang berarti harga-harga pada toko yang berada pada data tidak menyebar terlalu jauh. Sedangkan pada jumlah produk terjual standar deviasinya ialah 2412 yang artinya penyebaran data terkait jumlah produk terjual cukup tinggi. Sedangkan pada standar deviasi penyebaranya sangat rendah karena standar deviasinya ialah 0.13, bahkan untuk ukuran rating yang jaraknya 1 sampai 5, std bernilai 0,13 menandakan pesebaran data sangat rendah.
Nilai skewness pada jumlah harga dan jumlah produk terjual sangat tinggi, yang artinya pesebaran sangat condong kearah kiri atau mendekati nol yang mengartikan terdapat data-data toko yang penjualanya atau harganya lebih tinggi dari rata-rata. Sedangkan pada rating skewnessnya menunjukan angka negatif yaitu negatif 2.24 yang artinya sebagian besar toko tersebut memiliki rating yang tinggi. Pada skala rating 1 sampai 5, maka banyak toko yang mendapat rating mendekati 5 dan kurva pesebaran cukup condong ke kanan.
Ketiga data tersebut memiliki nilai kurtosis diatas 0 yang berarti merupakan leptokurtic dimana puncaknya cukup tinggi dan banyak data outliersnya

#### Potensi minimum maksimum

In [49]:
dfseblak['total_income'] = dfseblak['harga']*dfseblak['total_terjual']

dfseblak.head(5)

,nama_produk,harga,nama_toko,lokasi,total_terjual,rating,total_income
3,seblak instan jeletot jajanan hits bandung 1 p...,4180,tokomakanankalimantan,Jakarta Barat,1000,4.8,4180000
4,"Kerupuk Seblak Rafael Viral - PEDAS DJ, 125 GRAM",11640,Golden Cakery,Jakarta Pusat,25,4.8,291000
6,KERUPUK SEBLAK MENTAH ANEKA WARNA,11500,jajanangarut19,Jakarta Selatan,100,4.9,1150000
7,Seblak Instan Ceu Nthien Khas Bandung Rasana N...,15000,Central Seblak Nusantara,Tangerang Selatan,1000,5.0,15000000
8,seblak frozen,10400,pelangi girls naget,Jakarta Timur,2000,4.9,20800000


In [50]:
std = dfseblak['total_income'].std()

n = len(dfseblak)

low, up = stats.norm.interval(0.95,loc=dfseblak['total_income'].mean(),scale=std/np.sqrt(n))

print('Batas Bawah :',round(low))
print('Batas Atas :',round(up))

Batas Bawah : 7788764
Batas Atas : 10605993


pada kasus ini, pada soal tugas total terjual dianggap sebagai penjualan bulanan yang artinya total income dianggap sebagai penghasilan bulanan

sehingga anggapnya batas bawah dan batas atas nilai confidence pada pendapatan bulanan ialah 7,7 juta dan 10,6 juta

jabodetabek dan luar jabodetabek

In [51]:
dfseblak['lokasi'].unique()

array(['Jakarta Barat', 'Jakarta Pusat', 'Jakarta Selatan',
       'Tangerang Selatan', 'Jakarta Timur', 'Kab. Bandung', 'Kab. Bogor',
       'Bandung', 'Surabaya', 'Jakarta Utara', 'Kab. Bekasi',
       'Kab. Tangerang', 'Surakarta', 'Kab. Garut', 'Medan', 'Depok',
       'Kab.Ciamis', 'Tangerang', 'Cimahi', 'Bekasi', 'Kab. Mojokerto',
       'Semarang'], dtype=object)

In [52]:
jabodetabek = '|'.join(["Jakarta", "Tangerang", "Depok", "Bogor", "Bekasi"])

dfseblak['wilayah'] = np.where(dfseblak['lokasi'].str.contains(jabodetabek, case=False) , "jkt", "luar")

dfseblak.head(10)

,nama_produk,harga,nama_toko,lokasi,total_terjual,rating,total_income,wilayah
3,seblak instan jeletot jajanan hits bandung 1 p...,4180,tokomakanankalimantan,Jakarta Barat,1000,4.8,4180000,jkt
4,"Kerupuk Seblak Rafael Viral - PEDAS DJ, 125 GRAM",11640,Golden Cakery,Jakarta Pusat,25,4.8,291000,jkt
6,KERUPUK SEBLAK MENTAH ANEKA WARNA,11500,jajanangarut19,Jakarta Selatan,100,4.9,1150000,jkt
7,Seblak Instan Ceu Nthien Khas Bandung Rasana N...,15000,Central Seblak Nusantara,Tangerang Selatan,1000,5.0,15000000,jkt
8,seblak frozen,10400,pelangi girls naget,Jakarta Timur,2000,4.9,20800000,jkt
9,Seblak Instan Pedas Home Made,3500,the Dhecip,Tangerang Selatan,2000,4.8,7000000,jkt
10,SEBLAK INSTANT TERMURAH,3999,Pusat Cuankie & Cemilan Frozen,Jakarta Timur,10000,4.8,39990000,jkt
11,Bumbu Seblak 200 g Dengan Kerupuk,33000,MSH Rempah,Jakarta Selatan,70,5.0,2310000,jkt
12,KERUPUK MENTAH SERBA 5000 BAHAN SEBLAK,6000,GK_Snack,Kab. Bandung,500,5.0,3000000,luar
13,Seblak Instan Pedas Home Made,3500,the Dhecip,Tangerang Selatan,2000,4.8,7000000,jkt


In [19]:
rata2_dijkt = dfseblak[dfseblak['wilayah'].isin(["jkt"])]['harga'].mean()
rata2_luarjkt = dfseblak[dfseblak['wilayah'].isin(["luar"])]['harga'].mean()
print(f"Rata-rata harga didalam jabodetabek ialah {round(rata2_dijkt)}\nSedangkan rata-rata harga diluar ialah {round(rata2_luarjkt)}")


Rata-rata harga didalam jabodetabek ialah 14483
Sedangkan rata-rata harga diluar ialah 16273


secara rata-rata berbeda, namun pada untuk menentukan apakah berkaitan atau tidak maka akan dilakukan uji korelasi

In [22]:
corr_rho, pval_s = stats.spearmanr(dfseblak['harga'], dfseblak['wilayah'])
print(f"rho-correlation: {corr_rho:.2f}\np-value: {pval_s}")

rho-correlation: 0.14
p-value: 0.011707806474975699


H0: tidak ada hubungan antara harga dengan lokasi di dalam/luar jabodetabek
H1: ada hubungan antara harga dengan lokasi di dalam/luar jabodetabek

karena pValue lebih rendah dari 0.05, maka H0 ditolak dan variabel harga memiliki pengaruh terhadap lokasi di dalam/luar jabodetabek. Namun pengaruh tersebut cukup kecil dan tidak terlalu signifikan

In [23]:
corr_rho, pval_s = stats.spearmanr(dfseblak['harga'], dfseblak['rating'])
print(f"rho-correlation: {corr_rho:.2f}\np-value: {pval_s}")

rho-correlation: 0.40
p-value: 2.4769796558337766e-14


H0: tidak ada hubungan antara harga dengan rating
H1: ada hubungan antara harga dengan rating

pValue cukup rendah dab jauh dari 0.05 sehingga H0 ditolak, variabel harga dan rating sangat berkaitan

# E. Conclusion

- harga di jabodetabek dan diluar jabodetabek berbeda namun tidak terlalu jauh
- harga sangat berpengaruh pada rating, sehingga dalam misi menaikan rating, perlu diperhatikan juga harga yang dijual
- rata-rata rating dari toko yang berhasil di scraping datanya ialah 4.87 sehingga dalam persaingan ratingnya sangat perlu untuk ditingkatkan
- terdapat 335 produk lain dari pencarian dengan keyword "seblak" di tokopedia yang menjadi pesaing
- skewnes dari total produk terjual condeng ke sebelah kiri atau mendekati 0, yang artinya banyak produk dalam data yang memiliki penjualan yang lebih rendah dibandingkan dengan outliers ataupun produk yang berhasil menjual dalam jumlah tinggi
- lower bound dan upper bound dalam confidence level dari data yang dianggap penjualan perbulan (total terjual dikali harga namun dianggap sebagai penjualan pernulan) ialah berkisar dari 7 juta hingga 10 juta, yang artinya bila tidak mampu menjadi outliers kanan maka penghasilanya kurang lebih segitu